# Table 5.1: Network Flow Extraction

In [1]:
import pandas as pd
import os
from common import rename_attack_ddos2019
from common import rename_attack_ids2017
from common import rename_attack_fhswfcnl
from common import rename_attack

In [2]:
header = "IPV4_SRC_ADDR,IPV4_DST_ADDR,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,FLOW_DURATION_MILLISECONDS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,DURATION_IN,DURATION_OUT,MIN_TTL,MAX_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,RETRANSMITTED_OUT_PKTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE"
header_length = len(header.split(","))

In [3]:
def read_traffic_files(path):
    result_list = []
    for filename in os.listdir(path):
        filepath = path + "/" + filename
        if filename.startswith("00.flows"):
            # Fehlerhafter Header in org/org_cicddos2019/flows/01-12-complete/00.flows 38
            # und org/org_cicddos2019/flows/01-12-complete/00.flows 52
            if dataset == "01-12" and (filename.endswith("flows 38") or filename.endswith("flows 52")):
                pass
            else:
                result_list.append(filepath)
    return result_list


def get_flow_dict(df_count, files):
    for filepath in files:
        with open(filepath, "r") as f:
            for line in f.readlines():
                length = len(line.split(","))
                if not line[0].isalpha():
                    df_count.setdefault(length, 0)
                    df_count[length] += 1
    return df_count

In [4]:
ddos_dict = dict()
for dataset in ["01-12", "03-11"]:
    flow_path = f"../org/org_cicddos2019/flows/{dataset}-complete"
    traffic_files = read_traffic_files(flow_path)
    ddos_dict = get_flow_dict(ddos_dict, traffic_files)

sum(ddos_dict.values()) - ddos_dict[header_length], ddos_dict[header_length]

(50663864, 880679)

In [5]:
ids_dict = dict()
csv_path = "../org/org_cic_ids_17/csv"
for dataset in ["friday", "monday", "thursday", "tuesday", "wednesday"]:
    flow_path = f"../org/org_cicids2017/flows/{dataset}"
    traffic_files = read_traffic_files(flow_path)
    ids_dict = get_flow_dict(ids_dict, traffic_files)

sum(ids_dict.values()) - ids_dict[header_length], ids_dict[header_length]

(769233, 1463228)

In [6]:
folders = ["apache",
"bo",
"calc",
"ftp",
"nmap",
"rev",
"smb",
"smbdos",
"telnet",
"ts"]
swf_dict = dict()
for item in folders:
    flow_path = f"../org/org_fhswfcnl/{item}/flows"
    traffic_files = read_traffic_files(flow_path)
    swf_dict = get_flow_dict(swf_dict, traffic_files)

sum(swf_dict.values()) - swf_dict[header_length], swf_dict[header_length]

(101346, 286801)

In [7]:
count_uncorrupted = [ids_dict[header_length], "",ddos_dict[header_length], "",  swf_dict[header_length]]
count_total = [sum(ids_dict.values()),"",sum(ddos_dict.values()),"",sum(swf_dict.values())]
count_total, count_uncorrupted

([2232461, '', 51544543, '', 388147], [1463228, '', 880679, '', 286801])

### Labels

In [8]:
df_ids = pd.read_csv("../data/transformation/cicids2017_labeled.csv")
df_ddos = pd.read_csv("../data/transformation/cicddos2019_labeled.csv")
df_swf = pd.read_csv("../data/transformation/fhswfcnl_labeled.csv")
df_ids["Attack"] = df_ids["Label"].apply(rename_attack_ids2017)
df_ddos["Attack"] = df_ddos["Label"].apply(rename_attack_ddos2019)
df_swf["Attack"] = df_swf["Label"].apply(rename_attack_fhswfcnl)

In [9]:
df = pd.read_parquet(f"../data/netflow_sample.parquet")
all_attacks = sorted([rename_attack(x) for x in df["Attack"].unique()])

In [10]:
attack_ids = {key:0 for key in all_attacks}
count_dict = df_ids["Attack"].value_counts().to_dict()
for key,value in count_dict.items():
    attack_ids[key] = value
attack_ids = pd.DataFrame.from_dict(attack_ids, orient="index")
attack_ids

,0
Benign,1180289
Bot,1666
DDoS,2267
DoS,261672
Infiltration,7
Injection,0
Password,3939
Scanning,158806
XSS,7


In [11]:
org_ids = df_ids.groupby("Attack")["Label"].unique()
org_ids

Attack
Benign                                                   [BENIGN]
Bot                                                         [Bot]
DDoS                                                       [DDoS]
DoS             [DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Do...
Infiltration                                       [Infiltration]
Password        [Web Attack  Brute Force, FTP-Patator, SSH-Pa...
Scanning                                               [PortScan]
XSS                                            [Web Attack  XSS]
Name: Label, dtype: object

In [12]:
column_ids = attack_ids.join(org_ids)
column_ids

,0,Label
Benign,1180289,[BENIGN]
Bot,1666,[Bot]
DDoS,2267,[DDoS]
DoS,261672,"[DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Do..."
Infiltration,7,[Infiltration]
Injection,0,NaN
Password,3939,"[Web Attack  Brute Force, FTP-Patator, SSH-Pa..."
Scanning,158806,[PortScan]
XSS,7,[Web Attack  XSS]


In [13]:
attack_ddos = {key:0 for key in all_attacks}
count_dict = df_ddos["Attack"].value_counts().to_dict()
for key,value in count_dict.items():
    attack_ddos[key] = value
attack_ddos = pd.DataFrame.from_dict(attack_ddos, orient="index")
attack_ddos

,0
Benign,48758
Bot,0
DDoS,833003
DoS,0
Infiltration,0
Injection,0
Password,0
Scanning,0
XSS,0


In [14]:
org_ddos = df_ddos.groupby("Attack")["Label"].unique()
org_ddos

Attack
Benign                                             [BENIGN]
DDoS      [Syn, UDP-lag, DrDoS_SNMP, TFTP, WebDDoS, DrDo...
Name: Label, dtype: object

In [15]:
org_ddos["DDoS"] = sorted(org_ddos["DDoS"])
org_ddos["DDoS"]

['DrDoS_DNS',
 'DrDoS_LDAP',
 'DrDoS_MSSQL',
 'DrDoS_NTP',
 'DrDoS_NetBIOS',
 'DrDoS_SNMP',
 'DrDoS_SSDP',
 'DrDoS_UDP',
 'LDAP',
 'MSSQL',
 'NetBIOS',
 'Portmap',
 'Syn',
 'TFTP',
 'UDP',
 'UDP-lag',
 'UDPLag',
 'WebDDoS']

In [16]:
column_ddos = attack_ddos.join(org_ddos)
column_ddos

,0,Label
Benign,48758,[BENIGN]
Bot,0,NaN
DDoS,833003,"[DrDoS_DNS, DrDoS_LDAP, DrDoS_MSSQL, DrDoS_NTP..."
DoS,0,NaN
Infiltration,0,NaN
Injection,0,NaN
Password,0,NaN
Scanning,0,NaN
XSS,0,NaN


In [17]:
attack_swf = {key:0 for key in all_attacks}
count_dict = df_swf["Attack"].value_counts().to_dict()
for key,value in count_dict.items():
    attack_swf[key] = value
attack_swf = pd.DataFrame.from_dict(attack_swf, orient="index")
attack_swf

,0
Benign,2579
Bot,0
DDoS,0
DoS,2048
Infiltration,963
Injection,0
Password,110740
Scanning,12978
XSS,0


In [18]:
df_swf.groupby("Attack")["Label"].unique()

Attack
Benign                [Benign]
DoS                      [DoS]
Infiltration    [Infiltration]
Password            [Password]
Scanning               [Recon]
Name: Label, dtype: object

In [19]:
count_idenfiable = [sum(attack_ids[0]), "", sum(attack_ddos[0]), "", sum(attack_swf[0])]
count_idenfiable

[1608653, '', 881761, '', 129308]

In [20]:
final = pd.concat([column_ids,column_ddos,attack_swf], axis=1)
final

,0,Label,0,Label,0
Benign,1180289,[BENIGN],48758,[BENIGN],2579
Bot,1666,[Bot],0,NaN,0
DDoS,2267,[DDoS],833003,"[DrDoS_DNS, DrDoS_LDAP, DrDoS_MSSQL, DrDoS_NTP...",0
DoS,261672,"[DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Do...",0,NaN,2048
Infiltration,7,[Infiltration],0,NaN,963
Injection,0,NaN,0,NaN,0
Password,3939,"[Web Attack  Brute Force, FTP-Patator, SSH-Pa...",0,NaN,110740
Scanning,158806,[PortScan],0,NaN,12978
XSS,7,[Web Attack  XSS],0,NaN,0


In [21]:
row_total = pd.DataFrame(count_total, columns=["total"]).transpose()
row_total.columns = final.columns
row_uncorrupted = pd.DataFrame(count_uncorrupted, columns=["uncorrupted"]).transpose()
row_uncorrupted.columns = final.columns
row_identifiable = pd.DataFrame(count_idenfiable, columns=["identifiable"]).transpose()
row_identifiable.columns = final.columns
final_with_flows = pd.concat([final, row_identifiable, row_uncorrupted, row_total], axis=0)
final_with_flows

,0,Label,0,Label,0
Benign,1180289,[BENIGN],48758,[BENIGN],2579
Bot,1666,[Bot],0,NaN,0
DDoS,2267,[DDoS],833003,"[DrDoS_DNS, DrDoS_LDAP, DrDoS_MSSQL, DrDoS_NTP...",0
DoS,261672,"[DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Do...",0,NaN,2048
Infiltration,7,[Infiltration],0,NaN,963
Injection,0,NaN,0,NaN,0
Password,3939,"[Web Attack  Brute Force, FTP-Patator, SSH-Pa...",0,NaN,110740
Scanning,158806,[PortScan],0,NaN,12978
XSS,7,[Web Attack  XSS],0,NaN,0
identifiable,1608653,,881761,,129308


In [22]:
final_with_flows.columns = ["# flows", "original labels","# flows", "original labels", "# flows"]
final_with_flows.to_csv("../data/tables/flow_summary.csv")